In [2]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5fff342ea51d49e23402683c410592072d5487798e3d888065a12dfdfa0cfeaf85aa08c4f9628b97149f851b24e02d0bb1c19821c2f480b284'

In [768]:
import re
import heapq
import numpy as np
import math
import scipy
from scipy.linalg import LinAlgError
from functools import cache
import itertools as it

In [4]:
from aocd import submit
from aocd.models import Puzzle
day = 10
year = 2025
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [5]:
# test_data = """"""
# data_test = test_data.splitlines()
data_test = puzzle.examples[0].input_data.splitlines()
data_test

['[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}',
 '[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}',
 '[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}']

In [6]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(166,
 ['[.#.####] (0,3,5,6) (1,3,4,5) (1,4,5,6) (2,3,5) (1,6) (0,1,4,6) (1,3) (0,4) (0,4,6) {37,65,17,38,73,53,58}',
  '[#.#.#.##.] (0,1,4,7) (1,3,6) (2,4,5,7,8) (5,7) (0,1,2,3,4,5,6,8) (0,1,3,5,6,7,8) (1,4,5,6) {39,71,25,42,54,59,59,42,40}',
  '[#.##.#.#] (0,3,6,7) (0,1,2,3,7) (0,3,4,5) (0,2,3,7) (1,3,4,5) (2,5,6) (0,1,2,4,6) {30,20,15,42,28,31,25,18}',
  '[#.#.] (0,2) (1,2,3) {110,10,120,10}',
  '[##.###] (1,4,5) (0,2,4) (0,2,3,4) (0,1,4,5) (2,4) {139,148,14,5,162,148}',
  '[.#..###.#] (0,3) (1,2,7,8) (2,3,5,6) (0,1,7) (0,1,2,6,7) (1,2,3,5,6,7) (0,1,4,5,6,7,8) (2,5,6,8) (2,5) (1,4,6) {52,79,56,36,31,63,59,68,40}',
  '[####.] (0,2,4) (0,2,3) (0,2,3,4) (0,1,2) (0,4) {45,12,26,11,24}',
  '[.#.#.] (1,2,4) (0,3,4) (0,1,4) (1,2,3) {119,24,22,128,130}',
  '[.##.#] (1,3) (0,2,4) (0,2) (2,3,4) {25,9,41,25,35}',
  '[#.#.] (0,3) (2,3) (1) (0,1) {22,119,1,7}'])

In [7]:
def parse(line):
    lights, *switches, joltages = line.split()
    lights = tuple([{'.': 0, '#': 1}[c] for c in lights if c in '.#'])
    switches = [tuple(map(int, s[1:-1].split(','))) for s in switches]
    joltages = tuple(map(int, joltages[1:-1].split(',')))
    return lights, switches, joltages

def solve(line):
    target_lights, switches, joltages = parse(line)
    # Implement solution here
    initial_state = (0, tuple([0]*len(target_lights)))
    stack = [initial_state]
    seen = {}
    while stack:
        N, lights = heapq.heappop(stack)
        if lights in seen and seen[lights] <= N:
            continue
        seen[lights] = N
        if lights == target_lights:
            return N
        for switch in switches:
            new_lights = list(lights)
            for idx in switch:
                new_lights[idx] ^= 1
            heapq.heappush(stack, (N+1, tuple(new_lights)))
    return None
    

In [8]:
# presses = [solve(dat) for dat in data_test] 
presses = [solve(dat) for dat in data] 
print(sum(presses))

422


In [37]:
puzzle.answer_a = str(sum(presses))

That's the right answer!  You are one gold star closer to decorating the North Pole. [Continue to Part Two]


# Part 2

In [910]:
def gen_combinations(max_press, pushes, A, orig_A, b, orig_b, allow_negative_b=False):
    # get next button to iterate
    # print(pushes)
    chosen = np.where(pushes >= 0)[0]
    new_A = A.copy()
    new_b = b.copy()          
    # print('###')
    # print(pushes, chosen, new_A)
    if not allow_negative_b:
        if np.any((A @ (pushes * (pushes > 0))) > b):
            return []
    
    if np.any(chosen):
        for idx in chosen:
            if np.any(new_A[:, idx]):            
                # print(chosen, idx)      
                new_b -= new_A[:, idx] * pushes[idx]
                if np.any(new_b < 0) and not allow_negative_b:
                    return []
                new_A[:, idx] = 0
                
    # print(pushes, chosen, new_A)
    for i, row in enumerate(new_A):
        if (row == 0).sum() == len(row) - 1:
            new_pushes = pushes.copy()
            idx = np.where(row != 0)[0][0]
            if new_pushes[idx] != -1:
                continue
            new_pushes[idx] = np.round(new_b[i] / row[idx], 2)
            if new_pushes[idx] < 0:
                # print('!!!')
                # print(idx, new_pushes, new_pushes[idx])
                return []
            # print(new_b)
            # print(f"set: [{i} / {idx}] {new_pushes[idx]} | {new_pushes} | {new_b[i]} / {row[idx]} {new_b[i] / row[idx]}")
            return gen_combinations(max_press, new_pushes, new_A, orig_A, new_b, orig_b, allow_negative_b=allow_negative_b)
                    
    todo = np.where(pushes == -1)[0]
    combs = []
    if len(todo):
        idx = todo[-1]
        # print(f"exe: [{idx}]: {max_press[idx]+1}")
        for i in range(max_press[idx]+1):
            new_pushes = pushes.copy()
            new_pushes[idx] = i
            # print("set: ", idx, i)
            combs += gen_combinations(max_press, new_pushes, A, orig_A, b, orig_b, allow_negative_b=allow_negative_b)
    else:
        if np.all(orig_A @ pushes == orig_b):
            # print("solution: ", pushes)
            # return [pushes]
            combs.append(pushes)
        else:
            # print((orig_A @ pushes) - orig_b, pushes)
            # print()
            pass
    return combs

def forward_substitute(m, y):
    v = np.array([0 for i in range(len(m))])
    for i in range(len(m)):
        v[i] = (y[i] - v.dot(m[i]))#/float(m[i][i])
    return v

def get_switch_matrix(dat):
    _, switches, target_joltages = parse(dat)

    smat = np.zeros((len(target_joltages), len(switches)), dtype=int)
    joltages = np.array(target_joltages, dtype=int)
    
    max_press = []
    for i, switch in enumerate(switches):
        for idx in switch:
            smat[idx, i] = 1
        max_press.append(min(joltages[smat[:, i] == 1]))
        
    return smat, joltages, max_press


def solve_lu(dat):
    _, switches, target_joltages = parse(dat)

    smat = np.zeros((len(target_joltages), len(switches)), dtype=int)
    joltages = np.array(target_joltages, dtype=int)

    max_press = []
    for i, switch in enumerate(switches):
        for idx in switch:
            smat[idx, i] = 1
        max_press.append(min(joltages[smat[:, i] == 1]))
    
    print(max_press, dat)
    
    try:
        P, L, U = scipy.linalg.lu(smat, permute_l=False)
        # print(P@smat)
        # print((L@U))
        # print('XXXX')
        j = P @ joltages
        # print(L)
        # print(U)
        U[np.abs(U) < 1e-9] = 0
        L = P@L
        L[np.abs(L) < 1e-9] = 0
        m, n = L.shape
        y = None
        solutions = []
        if m > n:
            for rows in it.combinations(range(m), n):
                if np.linalg.matrix_rank(L[rows, :]) == n:
                    y = np.linalg.solve(L[rows, :], joltages[list(rows)])
                    try:
                        solutions = [np.linalg.solve(U, y)]
                    except LinAlgError:
                        pass
                    break
        if not len(solutions):
            y = np.linalg.solve(L, joltages)
            # print(y)
            # print(smat.shape)
            solutions = gen_combinations(max_press, np.array([-1] * len(switches)), U, smat.astype(int), y, joltages.astype(int), allow_negative_b=True)  
            solutions[0]
    except(LinAlgError, IndexError) as e:
        # raise(e)
        print('LinAlgError')
        solutions = gen_combinations(max_press, np.array([-1] * len(switches)), smat.astype(int), smat.astype(int), joltages.astype(int), joltages.astype(int), allow_negative_b=False)
    # return solutions
    # print(len(solutions))
 
    return sorted(solutions, key=sum)[0]
    


In [911]:
# solve_lu(data[11]) # 24
solve_lu(data[102])

[np.int64(45), np.int64(42), np.int64(46), np.int64(42), np.int64(42), np.int64(45), np.int64(46), np.int64(57), np.int64(62), np.int64(42), np.int64(42)] [#.##..####] (0,1,2,4,6,9) (1,3,4,5,6,7,8,9) (2,7) (0,2,3,4,5,6) (0,1,3,4,5,8,9) (0,1,2,4,6,8) (4,5,7,8) (1,2,5) (1,4,8) (0,1,2,3,5,6,7,8,9) (1,3,5) {45,77,57,42,73,60,57,46,62,46}


array([12, 20, 11,  5,  8, 14,  9,  9,  5,  6,  3])

In [912]:
res = [solve_lu(dat) for dat in data]

[np.int64(37), np.int64(38), np.int64(53), np.int64(17), np.int64(58), np.int64(37), np.int64(38), np.int64(37), np.int64(37)] [.#.####] (0,3,5,6) (1,3,4,5) (1,4,5,6) (2,3,5) (1,6) (0,1,4,6) (1,3) (0,4) (0,4,6) {37,65,17,38,73,53,58}
[np.int64(39), np.int64(42), np.int64(25), np.int64(42), np.int64(25), np.int64(39), np.int64(54)] [#.#.#.##.] (0,1,4,7) (1,3,6) (2,4,5,7,8) (5,7) (0,1,2,3,4,5,6,8) (0,1,3,5,6,7,8) (1,4,5,6) {39,71,25,42,54,59,59,42,40}
[np.int64(18), np.int64(15), np.int64(28), np.int64(15), np.int64(20), np.int64(15), np.int64(15)] [#.##.#.#] (0,3,6,7) (0,1,2,3,7) (0,3,4,5) (0,2,3,7) (1,3,4,5) (2,5,6) (0,1,2,4,6) {30,20,15,42,28,31,25,18}
[np.int64(110), np.int64(10)] [#.#.] (0,2) (1,2,3) {110,10,120,10}
[np.int64(148), np.int64(14), np.int64(5), np.int64(139), np.int64(14)] [##.###] (1,4,5) (0,2,4) (0,2,3,4) (0,1,4,5) (2,4) {139,148,14,5,162,148}
LinAlgError
[np.int64(36), np.int64(40), np.int64(36), np.int64(52), np.int64(52), np.int64(36), np.int64(31), np.int64(40), 

In [915]:
tot = sum([sum(r) for r in res])
int(tot)

16361

In [916]:
puzzle.answer_b = int(tot)

That's the right answer!  You are one gold star closer to decorating the North Pole.You have completed Day 10! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [922]:
### Testing below

In [925]:
_, switches, target_joltages = parse(data[0])
switches, joltages

([(0, 3, 5, 6),
  (1, 3, 4, 5),
  (1, 4, 5, 6),
  (2, 3, 5),
  (1, 6),
  (0, 1, 4, 6),
  (1, 3),
  (0, 4),
  (0, 4, 6)],
 array([45, 77, 57, 42, 73, 60, 57, 46, 62, 46]))

In [ ]:
def solve2_cache(line):
    _, switches, target_joltages = parse(line)
    # print(switches, target_joltages)
    
    @cache
    def find_min_pushes(state):
        pushes = []
        if sum(state) == 0:
            return 0
        # print(state)
        if min(state) < 0:
            return int(1e6)
        
        for sw in switches:
            new_state = tuple([jolt-1 if i in sw else jolt for i, jolt in enumerate(state)])
            # print(new_state)
            pushes.append(find_min_pushes(new_state))
        return min(pushes) + 1
    
    return find_min_pushes(tuple(target_joltages))

In [962]:
solve2_cache(data_test[0])

10

In [ ]:
# https://topaz.github.io/paste/#XQAAAQDYAgAAAAAAAAAzHIoib6p4r/McpYgEEgWhHoa5LSRMkVi92ASWXgRJn/53djPNKV5A7wufzTb/3LT8y9Pwd2fanbtFY6dbBckkDi2CLqxJgNV64TnM7AeC9eelyzmX9r8vZHrMlRXuPsQTyXOScK5wjcIXvvcF8YDQStF8lDVUncHNK/AQSgWEV/JGXRdCLVqr0B1O3IEIyM49eF/+9umeACBwCWEvA1zk81pXN6IrrxybsrTU/p24bZLRN+iWoaSEQOyQRXHd2FtotBmoaTIDeAcgvw4UsnLdRM/zfKQZnLHdXVAvxHg0/n7AKVfbypHwAoMKePamgWNSqDfAFpI4qHs0+Fa7ITNq8OxvRGhOSwuh9bzE3TOboMp+JiPiQP2crpqQDgolCXPBRyygwwjpxYQDzfcSEx643qs6CtCeE2yAJOnH7ut6hWdUoxdCAXHRuxNo8Pv5CtDk94OAnARKo8+y42jA5eLJ4D7fVBY49rqXET+IDuh/xtewnQ57fjv36tYkI7cvAf+V/T5R
scipy.optimize.linprog([1] * smat.shape[1], A_eq=smat, b_eq=joltages, integrality=1).fun

102.0

In [ ]:
# https://www.reddit.com/r/adventofcode/comments/1pity70/comment/nt8vlv9/
import collections, functools, itertools


def powerset(s): 
    for r in range(len(s)+1): yield from itertools.combinations(s,r)


def solve(line):
    lights, *buttons, demands = [g[1:-1] for g in line.split()]

    lights  = tuple(l=='#' for l in lights)
    buttons = tuple(set(eval(b+',')) for b in buttons)
    demands = tuple(eval(demands))

    options, output = collections.defaultdict(list), dict()
    for pressed in powerset(range(len(buttons))):
        supply = [len([1 for b in pressed if j in buttons[b]])
                         for j in range(len(demands))]
        parity = tuple(j%2 for j in supply)

        options[parity] += [pressed]
        output[pressed] = supply


    @functools.cache
    def opt(demands):
        if min(demands)  < 0: return 1e99
        if sum(demands) == 0: return 0

        answer = 1e99
        parity = tuple(j%2 for j in demands)

        for pressed in options[parity]:
            remain = tuple((j-s)//2 for j,s in zip(demands, output[pressed]))
            answer = min(answer, len(pressed) + 2*opt(remain))

        return answer

    return min(map(len, options[lights])), opt(demands)


print(*map(sum, zip(*map(solve, data))))

422 16361


In [963]:
### storage

In [ ]:
def solve_2(line):
    target_lights, switches, target_joltages = parse(line)
    # Implement solution here
    switches = sorted(switches, key=len, reverse=True)
    initial_state = (0, 0, tuple([0]*len(target_joltages)))
    stack = [initial_state]
    seen = {}
    res = []
    cnt = 0
    while stack:
        # print(stack)
        cnt += 1
        _, N, joltage = heapq.heappop(stack)
        skip = False
        for i, j in enumerate(joltage):
            if j > target_joltages[i]:
                skip = True
                break
            if skip:
                break
        if skip:
            continue
        

        for switch in switches:
            new_joltage = list(joltage)
            for idx in switch:
                new_joltage[idx] += 1
            
            new_joltage = tuple(new_joltage)
            
            # heapq.heappush(stack, (-sum(new_joltage), N+1, tuple(new_joltage)))
            if new_joltage == target_joltages:
                res.append(N+1)
            else:    
                if new_joltage in seen and seen[new_joltage] <= N+1:
                    continue
                
                for i, j in enumerate(new_joltage):
                    if target_joltages[i] < j:
                        break
                else:
                    diff  = sum(bi - ai for ai, bi in zip(new_joltage, target_joltages))
                    if diff == 0:
                        print('!!!')
                    heapq.heappush(stack, (diff, N+1, new_joltage))
                    seen[new_joltage] = N+1
        if len(res) >= 1 or cnt > 1e6:
            # print(stack[:3])
            # print(stack[-3:])
            # print(target_joltages)
            break
    # print('#: ', min(res))
    if stack:
        print(cnt, len(stack), len(seen), heapq.heappop(stack), heapq.heappop(stack))
    return min(res)